In [2]:
# Setup models

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
import os

from dotenv import load_dotenv

load_dotenv() ## loading the .env file

model = ChatOpenAI(model_name="gpt-4o")

#Setup the Embedding
embedding = OpenAIEmbeddings(
    model="text-embedding-3-small",
)


In [3]:
# Add Product Pricing function tool

import pandas as pd
from langchain_core.tools import tool

#Load the laptop product pricing CSV into a Pandas dataframe.
# this Pandas dataframe functions like RDBMS table
product_pricing_df = pd.read_csv("data/Laptop pricing.csv")
print(product_pricing_df)

@tool
def get_laptop_price(laptop_name:str) -> int :
    """
    This function returns the price of a laptop, given its name as input.
    It performs a substring match between the input name and the laptop name.
    If a match is found, it returns the pricxe of the laptop.
    If there is NO match found, it returns -1
    """

    #Filter Dataframe for matching names
    match_records_df = product_pricing_df[
                        product_pricing_df["Name"].str.contains(
                                                "^" + laptop_name, case=False)
                        ]
    #Check if a record was found, if not return -1
    if len(match_records_df) == 0 :
        return -1
    else:
        return match_records_df["Price"].iloc[0]

#Test the tool. Before running the test, comment the @tool annotation
#print(get_laptop_price("alpha"))
#print(get_laptop_price("testing"))

            Name  Price  ShippingDays
0  AlphaBook Pro   1499             2
1     GammaAir X   1399             7
2  SpectraBook S   2499             7
3   OmegaPro G17   2199            14
4  NanoEdge Flex   1699             2


In [4]:
# Add Product Retrieval tool

import sqlite3
from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma 
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the product featuers document.
loader=PyPDFLoader("./data/Laptop product descriptions.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)

#Create a vector store with Chroma
prod_feature_store = Chroma.from_documents(
    documents=splits, 
    embedding=embedding
)

get_product_features = create_retriever_tool(
    prod_feature_store.as_retriever(search_kwargs={"k": 1}),
    name="Get_Product_Features",
    description="""
    This store contains details about Laptops. It lists the available laptops
    and their features including CPU, memory, storage, design and advantages
    """
)

#Test the product feature store
#print(prod_feature_store.as_retriever().invoke("Tell me about the AlphaBook Pro") )

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [5]:
# Setup a Product QnA chatbot
# prebuilt create_react_agent doubles as both Agent and ChatBot

from langgraph.prebuilt import create_react_agent

#from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create a System prompt to provide a persona to the chatbot
system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about laptops sold by your company.
    To answer questions about laptops, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)

#Create a list of tools available
tools = [get_laptop_price, get_product_features]

#Create memory across questions in a conversation (conversation memory)
checkpointer=MemorySaver()

#Create a Product QnA Agent. This is actual a graph in langGraph. This creates Agent and Chatbot in one step.
product_QnA_agent=create_react_agent(
                                model=model, #LLM to use
                                tools=tools, #List of tools to use
                                state_modifier=system_prompt, #The system prompt
                                debug=False, #Debugging turned on if needed
                                checkpointer=checkpointer #For conversation memory
)

In [ ]:
#Setup chatbot
import uuid
#To maintain memory, each request should be in the context of a thread.
#Each user conversation will use a separate thread ID
#when running agent as a chatbot, we need to identify each conversation uniquely so that the Chatbot can track conversation history. thats why we need a config object
config = {"configurable": {"thread_id": uuid.uuid4()}}
# unique thread ID "thread_id": uuid.uuid4() is provided to link all prompts belonging to the same conversation together. This allows the chatbot to maintain context and memory across multiple interactions within the same conversation.

#Test the agent with an input
inputs = {"messages":[
                HumanMessage("What are the features and pricing for GammaAir?")
            ]}

#Use streaming to print responses as the agent  does the work.
#This is an alternate way to stream agent responses without waiting for the agent to finish
# in Request-response mode, the agent waits until the final response is ready then delivers all the results
# this is the streaming mode, the agent sends the results for intermediate steps as they become available, allowing us to see the agent's thought process and tool usage in real time. 
for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message=stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the features and pricing for GammaAir?
================================== Ai Message ==================================
Tool Calls:
  Get_Product_Features (call_fbBFjq7eaFi92F3gLzJ4qr2L)
 Call ID: call_fbBFjq7eaFi92F3gLzJ4qr2L
  Args:
    query: GammaAir
  get_laptop_price (call_VHIdLBf9PsE5iojAAOdW3ELV)
 Call ID: call_VHIdLBf9PsE5iojAAOdW3ELV
  Args:
    laptop_name: GammaAir


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


================================= Tool Message =================================
Name: get_laptop_price

1399
================================== Ai Message ==================================

The GammaAir X laptop combines impressive features and performance, making it ideal for users seeking both power and portability. Here are the key features and pricing of the GammaAir X:

**Features:**
- **Processor:** AMD Ryzen 7
- **Memory:** 32GB of DDR4 RAM
- **Storage:** 512GB NVMe SSD
- **Design:** Thin and light form factor

**Price:** $1399

This configuration offers high performance in a portable design, perfect for those who require a robust laptop on the go.


In [7]:
# Execute the Product QnA Chatbot
import uuid
#Send a sequence of messages to chatbot and get its response
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a laptop",
    "Give me a list of available laptop names",
    "Tell me about the features of  SpectraBook",
    "How much does it cost?",
    "Give me similar information about OmegaPro",
    "What info do you have on AcmeRight ?",
    "Thanks for the help"
]

#Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = product_QnA_agent.invoke(user_message,config=config)
    #Print the response
    print(f"AGENT : {ai_response['messages'][-1].content}")
    

----------------------------------------
USER : Hello
AGENT : Hello! How can I assist you today with your laptop inquiries?
----------------------------------------
USER : I am looking to buy a laptop
AGENT : Great! I can help you with information on the laptops we offer. Do you have specific features in mind, or would you like to know about the available options and their prices?
----------------------------------------
USER : Give me a list of available laptop names
AGENT : Here is a list of available laptop names:

1. **AlphaBook Pro** - A sleek ultrabook with powerful features for professionals.
2. **GammaAir X** - Offers high performance in a portable design.
3. **SpectraBook S** - Designed for power users with workstation-class specifications.
4. **OmegaPro G17** - A gaming powerhouse for the best gaming experience.
5. **NanoEdge Flex** - Additional details aren't provided, but it completes the list of available laptops.

If you are interested in any specific model or need detail

In [8]:
#conversation memory by user
def execute_prompt(user, config, prompt):
    inputs = {"messages":[("user",prompt)]}
    ai_response = product_QnA_agent.invoke(inputs,config=config)
    print(f"\n{user}: {ai_response['messages'][-1].content}")

#Create different session threads for 2 users
config_1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

#Test both threads
execute_prompt("USER 1", config_1, "Tell me about the features of  SpectraBook")
execute_prompt("USER 2", config_2, "Tell me about the features of  GammaAir")
execute_prompt("USER 1", config_1, "What is its price ?")
execute_prompt("USER 2", config_2, "What is its price ?")


USER 1: The SpectraBook S is designed for power users, featuring an Intel Core i9 processor, 64GB RAM, and a massive 2TB SSD. This workstation-class laptop is ideal for intensive tasks such as video editing and 3D rendering.

USER 2: The GammaAir X is a high-performance laptop featuring an AMD Ryzen 7 processor, 32GB of DDR4 memory, and a 512GB NVMe SSD. Its thin and light design makes it ideal for users who require both portability and performance.

USER 1: The price of the SpectraBook S is $2,499.

USER 2: The price of the GammaAir X is $1,399.
